## Initial Settings

In [1]:
import os
import numpy as np
import pandas as pd
import warnings

from google.colab import drive

warnings.filterwarnings('ignore')
drive.mount("/content/drive")

os.chdir("drive/MyDrive/competition/2022-AI-competition-Round1") # Local Path
os.listdir()

Mounted at /content/drive


['competition_data',
 'html',
 'submission',
 'playground.ipynb',
 'AutoML Baseline.ipynb',
 'Evaluator Module.ipynb',
 '2022-08-04 feature EDA.ipynb',
 "2022-08-05 LGB_train(hyeonbin's parameter tuning).ipynb",
 'model compare.ipynb',
 'Optuna Optimization.ipynb',
 '2022-08-06 Optimize ET.ipynb',
 '2022-08-06 Regressor model compare.ipynb',
 'catboost_info',
 'evaluator',
 '.git',
 '.gitignore',
 'README.md',
 'Updated CLF.ipynb',
 'GitHub Connection.ipynb',
 '2022-08-17 Ensemble Module.ipynb',
 '2022-08-11 Models(XGB_ET) Optimization.ipynb']

In [2]:
!pip install -r evaluator/requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 87 kB/s 
     |████████████████████████████████| 308 kB 68.9 MB/s 
     |████████████████████████████████| 81 kB 9.9 MB/s 
     |████████████████████████████████| 209 kB 24.0 MB/s 
     |████████████████████████████████| 78 kB 7.8 MB/s 
     |████████████████████████████████| 49 kB 5.4 MB/s 
     |████████████████████████████████| 147 kB 40.0 MB/s 
     |████████████████████████████████| 112 kB 49.6 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=3efad751289498574d1ebb2cbff6ef751ce4e2eee6ff61dd6d9505aaffd9b0e8
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [3]:
from evaluator.evaluator import Evaluator, Model

train_df = pd.read_csv('competition_data/train.csv')
test_df = pd.read_csv("competition_data/test.csv")
submission_df = pd.read_csv("competition_data/sample_submission.csv")

In [4]:
from evaluator.evaluator import Model, Evaluator, Optimizer
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.model_selection import KFold
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    mean_absolute_error,
)

class MyEvaluator(Evaluator):
  def __init__(self, *args, **kwargs):
    super().__init__(*args, **kwargs)

  def evaluate(self, metrics="all", train_acc=True):
    total_err = 0
    total_score = 0

    train_x=self.train_df.drop(['nerdiness'], axis=1)
    train_y=self.train_df['nerdiness']

    if metrics == "all":
      metrics = [
          "accuracy", "precision", "recall", 
          "f1-score", "roc_auc", "mae"
      ]
    metrics_functions_map = {
        "accuracy": accuracy_score,
        "precision": precision_score,
        "recall": recall_score,
        "f1-score": f1_score,
        "roc_auc": roc_auc_score,
        "mae": mean_absolute_error,
    }
    class_metrics = {
        "accuracy",
        "precision",
        "recall",
        "f1-score",
    }
    result_df = pd.DataFrame(
        columns = metrics + ["train_acc"]
    )

    for i, (train_index, val_index) in enumerate(self.kf.split(train_x)):
      X_train, X_test = train_x.loc[train_index], train_x.loc[val_index]
      y_train, y_test = train_y.loc[train_index], train_y.loc[val_index]

      self.model.fit(X_train, y_train, **self.fit_params)
      if self.model_type == 'rgr':
        predictions = self.model.predict(X_test)
      else:
        predictions = self.model.predict_proba(X_test)[:,1]

      row = {}
      for metric in metrics:
        if metric in class_metrics:
          score = metrics_functions_map[metric](
              y_test,
              np.array(predictions) > 0.5,
          )
        else:
          score = metrics_functions_map[metric](y_test, predictions)
        row[metric] = score
      result_df = result_df.append(
          row, ignore_index=True
      )

    result_df["fold"] = list(range(1, i+2))
    result_df = result_df.set_index("fold")

    ## add training accuracy
    mean = result_df.mean(axis=0)
    self.model.fit(train_x, train_y, **self.fit_params)
    if self.model_type == 'rgr':
      predictions = self.model.predict(train_x)
    else:
      predictions = self.model.predict_proba(train_x)[:,1]
    mean["train_acc"] = accuracy_score(
      np.array(predictions) > 0.5, 
      train_y,
    )
    result_df.loc["mean"] = mean

    return result_df

  def make_submission(self, test_df, submission_df):
    test_df = self.preprocess(test_df)

    # handle nan values
    imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
    imp = imp.fit(test_df)
    test_df = pd.DataFrame(
        imp.transform(test_df),
        columns = test_df.columns
    )
    
    preds = self.model.predict(test_df)
    submission_df["nerdiness"] = preds
    return submission_df


class MyModel(Model):
  def __init__(self, *args, **kwargs):
    super().__init__(*args, **kwargs)

  def optimize(self, initial_params, **kwargs):
    self.optimizer = MyOptimizer(
        self.train_df, 
        initial_params, 
        self.model_name,
        self.model_type,
    )
    best_params = self.optimizer.run(**kwargs)
    self.__init__(self.train_df, self.model_name, self.model_type, **best_params)

class MyOptimizer(Optimizer):
  def __init__(self, *args, **kwargs):
    super().__init__(*args, **kwargs)

  def objective(self, trial):
    ## Tuning Parmeters
    for param, dtype, value in self.initial_params:
      if dtype == "static":
        self.params[param] = value
      elif dtype == "int":
        self.params[param] = trial.suggest_int(param, *value)
      elif dtype == "float":
        self.params[param] = trial.suggest_uniform(param, *value)
      elif dtype == "log":
        self.params[param] = trial.suggest_loguniform(param, *value)
      elif dtype == "categorical":
        self.params[param] = trial.suggest_categorical(param, value)
      else:
        raise NameError("dtype must be one of ('static', 'int', 'float', 'log', 'categorical')")

    ## Objective Metric
    result_df = MyEvaluator(
        **MyModel(self.train_df, self.model_name, self.model_type, **self.params).get_model()
    ).run(train_acc=False)

    return result_df["roc_auc"]["mean"]

### XGB, ET Baseline

In [ ]:
model = MyModel(train_df, "xgb", "rgr")

In [ ]:
evaluator = MyEvaluator(
    **model.get_model()
)
evaluator.run()

[07:07:44] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:07:46] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:07:47] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:07:49] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:07:51] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


,accuracy,precision,recall,f1-score,roc_auc,mae,train_acc
fold,,,,,,,
1,0.730541,0.734673,0.814031,0.772319,0.801982,0.364868,NaN
2,0.730228,0.723350,0.817556,0.767573,0.809513,0.363450,NaN
3,0.731791,0.734829,0.810006,0.770588,0.803902,0.367782,NaN
4,0.737418,0.759422,0.797290,0.777895,0.800348,0.365465,NaN
mean,0.732495,0.738069,0.809721,0.772094,0.803936,0.365391,0.759143


In [ ]:
submission_df = evaluator.make_submission(test_df, submission_df)
submission_df.to_csv("submission/2022-08-17_XGB_base.csv", index=False)

In [ ]:
params={
    "criterion": "gini",
    "random_state": 42
}
evaluator = Evaluator(
    **Model(train_df, "et", "clf").get_model()
)
evaluator.run()

,accuracy,precision,recall,f1-score,roc_auc,mae,train_acc
fold,,,,,,,
1,0.788997,0.795155,0.840757,0.817321,0.872260,0.303267,NaN
2,0.781182,0.769788,0.853701,0.809576,0.876518,0.305080,NaN
3,0.774930,0.771678,0.845419,0.806867,0.865260,0.312729,NaN
4,0.787434,0.805454,0.832520,0.818763,0.872283,0.303807,NaN
mean,0.783135,0.785519,0.843099,0.813132,0.871580,0.306221,1.0


### Optuna Tuning

In [5]:
model = MyModel(train_df, "xgb", "rgr")
initial_params = (
    ("n_estimators", "int", (100, 1000)),
    ("objective", "static", "reg:squarederror"),
    ("average", "static", "micro"),
    ("learning_rate", "log", (1e-3, 1.0)),
    ("gamma", "int", (0, 5)),
    ("max_depth", "int", (5, 20)),
    ("lambda", "int", (0, 5)),
    ("alpha", "int", (0, 5)),
    ("subsample", "float", (0.7, 1))
)
model.optimize(initial_params, n_trials=50)

[I 2022-08-18 00:51:57,324] A new study created in memory with name: no-name-9ddc9974-d768-4841-b709-38fef9d9b7df


KeyboardInterrupt: ignored

In [ ]:
evaluator = Evaluator(
    **model.get_model()
)
evaluator.run()

In [ ]:
params = {
    "objective": "reg:squarederror",
    "average": "micro",
    'n_estimators': 746, 
    'learning_rate': 0.0181863245309935, 
    'gamma': 0, 
    'max_depth': 11, 
    'lambda': 1, 
    'alpha': 4, 
    'subsample': 0.7231465475519483
}
evaluator = Evaluator(
    **Model(train_df, "xgb", "rgr", **params).get_model()
)
evaluator.run()

,accuracy,precision,recall,f1-score,roc_auc,mae,train_acc
fold,,,,,,,
1,0.783995,0.793727,0.831292,0.812075,0.870709,0.288328,NaN
2,0.779619,0.773158,0.842800,0.806478,0.872918,0.291439,NaN
3,0.780244,0.781087,0.840360,0.809640,0.868822,0.295442,NaN
4,0.780556,0.800948,0.824390,0.812500,0.864985,0.291375,NaN
mean,0.781103,0.787230,0.834710,0.810173,0.869358,0.291646,1.0


In [ ]:
submission_df = evaluator.make_submission(test_df, submission_df)
submission_df.to_csv("submission/2022-08-11_XGB_optim_20.csv", index=False)

In [ ]:
import optuna
optuna.visualization.plot_param_importances(model.optimizer.opt)

In [ ]:
optuna.visualization.plot_optimization_history(model.optimizer.opt)

In [ ]:
optuna.visualization.plot_slice(model.optimizer.opt)

In [ ]:
model = Model(train_df, "lgbm", "rgr")
initial_params = (
    ("n_estimators", "int", (500, 5000)),
    ("objective", "static", "binary"),
    ("learning_rate", "log", (0.01, 1.0)),
    ("gamma", "int", (0, 5)),
    ("max_depth", "int", (10, 20)),
    ("lambda", "int", (1, 4)),
    ("alpha", "int", (1, 5)),
    ("subsample", "float", (0.8, 1))
)
model.optimize(initial_params, n_trials=500)